生成需要得伪数据:

In [ ]:
from mxnet import ndarray as nd
from mxnet import autograd as ag

num_inputs = 2
num_examples = 1000

# y[i] = 2 * x[i][0] -3.4 * x[i][1] + 4.2 + noise
w_true = [2, -3.4]
b_true = 4.2

X = nd.random_normal(shape = (num_examples, num_inputs))
y = w_true[0] * X[:, 0] + w_true[1] * X[:, 1] + b_true
y += .01 * nd.random_normal(shape=y.shape)

print (X[0], y[0])

不用gluon，只用ndarray（需要先跑伪数据）:

In [ ]:
import random

batch_size = 10

def data_iter():
    idx = list(range(num_examples))
    random.shuffle(idx)
    for i in range(0, num_examples, batch_size):
        j = nd.array(idx[i:min(i + batch_size,num_examples)])
        yield nd.take(X,j), nd.take(y,j)


w = nd.random_normal(shape=(num_inputs, 1))
b = nd.zeros((1,))
params = [w,b]

for param in params:
    param.attach_grad()


def net(X):
    return nd.dot(X, w) + b

def square_loss(yhat, y):
    return (yhat - y.reshape(yhat.shape)) ** 2

def SGD(params, lr):
    for param in params:
        param[:] = param - lr * param.grad

epochs = 5
learning_rate = .01

for e in range(epochs):
    total_loss = 0
    for data, label in data_iter():
        with ag.record():
            output = net(data)
            loss = square_loss(output, label)
        
        loss.backward()
        SGD(params, learning_rate)
        
        total_loss += nd.sum(loss).asscalar()
    print ("Epoch %d, average loss: %f" %(e, total_loss/num_examples))
        
    

只用gluon（需要先跑伪数据）:

In [ ]:
from mxnet import gluon

batch_size = 10
dataset = gluon.data.ArrayDataset(X, y)
data_iter = gluon.data.DataLoader(dataset, batch_size, shuffle=True)

net_g = gluon.nn.Sequential()
net_g.add(gluon.nn.Dense(1)) #1代表输出,input不要设置

net_g.initialize()
square_loss = gluon.loss.L2Loss()

trainer = gluon.Trainer(net_g.collect_params(), 'sgd', {'learning_rate':0.1})#不同点！！！

epochs = 5
learning_rate = .01

for e in range(epochs):
    total_loss = 0
    for data, label in data_iter:
        with ag.record():
            output = net_g(data)
            loss = square_loss(output, label)
        
        loss.backward()
        trainer.step(batch_size) #SGD(params, learning_rate)不同点！！！

        total_loss += nd.sum(loss).asscalar()
    print ("Epoch %d, average loss: %f" %(e, total_loss/num_examples))
